### Using Keras Tuner library

## To decide No of Hidden layers and no of neurons in a Neural Network

In [1]:
## Importing libraries

In [2]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [3]:
df= pd.read_csv('Real_Combine.csv')

In [4]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [5]:
x=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

### Hper paramters
  1. How many hidden layers we should use?
  2. How many neurons we should use in hidden layers?
  3. learning rate.

In [6]:
def build_model(hp):
    model=keras.Sequential()  ## Creatinng a ANN model
    for i in range(hp.Int('num_layers',2,20)): ## setting a range of layers from 2 to 20
        model.add(layers.Dense(units=hp.Int('units'+str(i),  ## setting a range of neurons in each hidden layer
                                           min_value=2,
                                           max_value=600,
                                           step=32),
                              activation='relu'))
        model.add(layers.Dense(1,activation='linear'))  ## output layers with  activation= linear since its a regression problem
        model.compile(
        optimizer=keras.optimizers.Adam(                 ## using optimizer adam
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),   ## taking any one learning rate 
        loss='mean_absolute_error',                             
        metrics=['mean_absolute_error'])
    return model
    

In [7]:
tuner = RandomSearch(
    build_model,   ## above function which automatically calls here
    objective='val_mean_absolute_error',     ## metrics
    max_trials=5,      ## total no of trails
    executions_per_trial=3,   ## no of execution in each trail
    directory='project',
    project_name='Air Quality Index')

INFO:tensorflow:Reloading Oracle from existing project project\Air Quality Index\oracle.json


In [8]:
tuner.search_space_summary()   

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 600, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 600, 'step': 32, 'sampling': None}


In [9]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [10]:
tuner.search(X_train, y_train,
             epochs=50,
             validation_data=(X_test, y_test))


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
num_layers        |10                |?                 
units0            |34                |?                 
learning_rate     |0.01              |?                 
units1            |514               |?                 



AttributeError: module 'tensorflow._api.v2.distribute' has no attribute 'TPUStrategy'